In [1]:
import mysql.connector
from nltk.tokenize import sent_tokenize
import import_ipynb
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import pandas as pd
from algoritma.MMR import *
from nltk.corpus import stopwords
import string
from termcolor import colored
import time
import csv
from nltk.tokenize import word_tokenize

importing Jupyter notebook from C:\Users\risma\Kuliah\MCA\Text-Summarization-News-Ariticles\algoritma\MMR.ipynb


# Deklarasi Connection Database

Membuat koneksi kedatabase lokal dengan database 'db-berita-mca'

In [2]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="db-berita-mca"  
) 

# Membuat Fungsi Steaming Use Sastrawi

Membuat stemmer pakai sastrawi 
Digunakan untuk melakukan steaming kalimat (menjadi kalimat dasar Menggunakan Library Sastrawi)

In [3]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Load Data Stopword Lib & File Internal

Mengambil data stopword bahasa indonesia yang pertama menggunakan file list dari stop word 917 dan menggunakan library nltk sebanyak panjang data 789

In [4]:
def load_stopWords():
    r = open("stopwordlist-indonesia.txt", "r")
    readTxt = r.read()
    idStopWord = readTxt.split() 
    return idStopWord

In [5]:
def load_stopWordsNLTK():
    stop_words = stopwords.words('indonesian')
    return stop_words

# Simpan hasil summary ke database

Menyimpan data yang telah dihasilakan melalui summary ke database dengan cara mengupdate data tersebut dan memasukan text summary pada field tersebut

In [21]:
def simpanData(mydb,summary,id,labeling_summary,score_summary,labeling_content,score_content):
    mycursor = mydb.cursor()
    id = int(id)
    sql = "UPDATE tb_berita SET summary = %s ,labeling_summary = %s ,score_summary = %s,labeling_content = %s,score_content = %s WHERE id = %s"
    param = (summary,labeling_summary,score_summary,labeling_content,score_content,id)
    mycursor.execute(sql, param)
    mydb.commit()

# Proses Text Preprocessing 

[tokenizeParagraftoKalimat, cleansingData, steamingData, indoStopWord]

Fungsi diatas merupakan fungsi yang digunakan untuk melakukan preprocessing article sebuah berita, penjelasan mengenai fungsi akan dijelaskan lebih lanjut

[tokenizeParagraftoKalimat]

berfungsi sebagai tokenize pargraf menjadi bagian perkalimat 

In [7]:
def tokenizeParagraftoKalimat(paragraf):
    tokenize = sent_tokenize(paragraf)
    return tokenize

[cleansingData]

berfungsi sebagai membersihkan data dari huruf tanda petik dan masih kata besar atau keceil

In [8]:
def cleansingData(text):
    hasil = re.sub('[^A-Za-z0-9 ]+', '', text)
    hasil = hasil.translate(str.maketrans("","",string.punctuation))
    return hasil

[steamingData]

berguna sebagai menjadikan kata dasar sebuah kalimat dan fungsi stemmer tersebut sekaligus menjadikan kalimat menjadi lower atau huruf kecil

In [9]:
def steamingData(text):
    steamingData = stemmer.stem(text)
    return steamingData

[indoStopWord]

berguna dalam menghilangkan kata sambung atau tidak diperlukan, ini berfungsi dlam menghitung kesamaan antar kalimat pada algoritma MMR agar lebih mudah dan mendapatkan hasil yang akurat (Lib NLTK & List Stopword)

In [10]:
def indoStopWord(stopNLTK, stopTxt, kalimat):
    hasilStopWord = []
    for kata in kalimat.split():
        if not kata in stopNLTK:
            if not kata in stopTxt:
                hasilStopWord.append(kata)
    return " ".join(hasilStopWord)

[hasilSummary]

fungsi yang berguna dalam menampilkan hasil summarization pada algoritma MMR

In [11]:
def hasilSummary(kalimatAsli, summarySet):
    hasil = []
    for sentence in summarySet:
        hasil.append(kalimatAsli [sentence].lstrip(' ')) 
    return " ".join(hasil)

[visualisasiSummaryOri]

menampilkan visualisasi yang sangat berguna dalam mengetahui kata mana saja yang dipakai atau ditampilkan pada summarization

In [12]:
def visualisasiSummaryOri(cleansingTeks,summarySet,kalimatAsli):
    for sentence in cleansingTeks:
        if sentence in summarySet:
            print(colored(kalimatAsli[sentence].lstrip(' '),'red'))
        else:
            print(kalimatAsli[sentence].lstrip(' '))

bisa dibilang sebagai code utama, dimana disini tempat menjalankan semua code dengan melakukan pemanggilan semua fungsi dan class algoritam MMR 

In [27]:
def summarization (data):
    for iteration, article in enumerate(data['article']):
        id = df['id'][iteration]
        tokenizeKalimat = tokenizeParagraftoKalimat(article)

        kalimat = []
        cleansingTeks = []
        kalimatAsli = {}

        for teks in tokenizeKalimat:
            kataDasar = steamingData(cleansingData(teks))
            dataFinal = indoStopWord(load_stopWordsNLTK(),load_stopWords(),kataDasar)
            cleansingTeks.append(dataFinal)
            kalimat.append(teks)
            kalimatAsli[dataFinal] = teks

        iteration = MMR()
        score = iteration.similarityScore(cleansingTeks)
        summarySet = iteration.calculateMMR(score,cleansingTeks)


        summary = hasilSummary(kalimatAsli,summarySet)
        polaritySummary,scoreSummary = analisisSummary(summary)
        content = " ".join(cleansingTeks)
        polarityContent,scoreContent = analisisSummary(content)
        simpanData(mydb,summary,id,polaritySummary,scoreSummary,polarityContent,scoreContent)
        print ('\nSummary dari Berita ID : {} (hasil teks yang diringkas):\n'.format(id))
        print(summary)
        print ('\nAnalisis Sentiment :'+polaritySummary)
        print ('\nDengan Score Sentiment :'+str(scoreSummary))  

        print ('=============================================================')
        print ('\Oringinal Article (Teks Asli):\n')
        visualisasiSummaryOri(cleansingTeks,summarySet,kalimatAsli)
        

[analisisSummary] fungsi yang berguna untuk melakukan analysis sentiment menggunkaan model lexicon da

In [24]:
def analisisSummary(text):
    lexicon_positive = dict()
    with open('data/lexicon_positive.csv', 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in reader:
            lexicon_positive[row[0]] = int(row[1])
        
    lexicon_negative = dict()
    with open('data/lexicon_negative.csv', 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in reader:
            lexicon_negative[row[0]] = int(row[1])
            
    tokonize = (word_tokenize(cleansingData(text)))
    hasilStopWord = []
    for kata in tokonize:
        if not kata in load_stopWordsNLTK():
            if not kata in load_stopWords():
                hasilStopWord.append(kata)
    hasilSteamingData = []
    for x in hasilStopWord:
        hasilSteamingData.append(steamingData(x))
    score = 0
    for word in hasilSteamingData:
        if (word in lexicon_positive):
            score = score + lexicon_positive[word]
    for word in hasilStopWord:
         if (word in lexicon_negative):
            score = score + lexicon_negative[word]
    polarity=''
    if (score > 0):
        polarity = 'positive'
    elif (score < 0):
        polarity = 'negative'
    else:
        polarity = 'neutral'
        
    return polarity,score

# Mengambil Data pada Database

Mengambil data pada database,data yang diambil hanya 2 kolom saja

In [25]:
mycursor = mydb.cursor()
mycursor.execute("SELECT id, content FROM tb_berita WHERE summary IS NULL")
myresult = mycursor.fetchall()
df = pd.DataFrame(myresult,columns=['id','article'])
df

,id,article
0,2440,"Jakarta, CNBC Indonesia - Ilmuwan China merili..."
1,2441,"Jakarta, CNBC Indonesia - Filantropi dan pendi..."
2,2442,"Jakarta, CNBC Indonesia - Harga nikel melemah ..."
3,2443,"Jakarta, CNBC Indonesia - Nilai tukar rupiah a..."
4,2444,"Jakarta, CNBC Indonesia - Indonesia telah meny..."
...,...,...
625,3065,"Kolonel P, satu dari tiga anggota TNI AD pelak..."
626,3066,Wali Kota Surakarta Gibran Rakabuming Raka men...
627,3067,"Menteri Agama (Menag), Yaqut Cholil Qoumas pri..."
628,3068,Jombang - Bupati Jombang Mundjidah Wahab buka ...


In [28]:
summarization(df)


Summary dari Berita ID : 2440 (hasil teks yang diringkas):

Obat itu sendiri telah menerima persetujuan darurat dari regulator obat top China, Administrasi Produk Medis Nasional. Mengingat statusnya yang menjadi obat antibodi, obat ini juga dapat digunakan untuk kelompok masyarakat yang tidak dapat menerima vaksin akibat alasan kesehatan tertentu. Jakarta, CNBC Indonesia - Ilmuwan China merilis obat antibodi Covid-19 terbarunya. Bahan-bahan tersebut didapatkan dari plasma pasien yang sudah sembuh.

Analisis Sentiment :positive

Dengan Score Sentiment :4
\Oringinal Article (Teks Asli):

Jakarta, CNBC Indonesia - Ilmuwan China merilis obat antibodi Covid-19 terbarunya.
Obat itu diklaim efektif dalam mengurangi risiko rawat inap atau kematian di antara pasien Covid-19 hingga 80%.
Obat itu dikembangkan oleh Fakultas Kedokteran Universitas Tsinghua.
Kepala penelitian obat, Zhang Linqi, mengatakan bahwa obat ramuan timnya itu ampuh dalam melawan varian termasuk Alpha, Delta dan Omicron.
Men


Summary dari Berita ID : 2444 (hasil teks yang diringkas):

SMS itu berisi link sertifikat vaksin, klik tautan untuk mengecek dan mendapatkan sertifikat vaksin. Bagi yang sudah memiliki akun bisa masuk dengan alamat email atau nomor terdaftar. Ketuk nama pengguna.8. Situs Web Peduli Lindungi Akses Peduli Lindungi bisa lewat website www.pedulilindungi.id dan dapat dilakukan melalui web browser atau ponsel. Tekan menu Sertifikat Vaksin7.

Analisis Sentiment :positive

Dengan Score Sentiment :25
\Oringinal Article (Teks Asli):

Jakarta, CNBC Indonesia - Indonesia telah menyelenggarakan program vaksinasi Covid-19 hampir setahun terakhir.
Bagi masyarakat yang sudah mendapatkan vaksin akan mendapatkan sertifikat di aplikasi PeduliLindungi Sertifikat ini jadi syarat untuk mengunjungi tempat-tempat publik.
Untuk menemukannya, ada berbagai cara mulai dari SMS, serta aplikasi dan web Peduli Lindungi.
Masyarakat akan mendapatkan sertifikat dengan jumlah dosis yang didapatkan.
Sejauh ini seluruh 


Summary dari Berita ID : 2448 (hasil teks yang diringkas):

Data WHO menunjukkan dari 47 negara yang telah memiliki izin penggunaan vaksin Sinovac, 16 di antaranya telah mengkonfirmasi kasus varian Omicron. Pusat Pencegahan dan Pengendalian Penyakit Eropa memperkirakan varian Omicron bisa menjadi varian dominan di Eropa dalam beberapa bulan. Perusahaan belum memastikan dan menolak untuk mengatakan kapan laporan studi tersebut akan dipublikasikan. Dari benua biru virus ini telah terdeteksi di Austria, Republik Cek, Estonia, Finlandia, Irlandia, Italia, Latvia, Liechtenstein, Luxembourg, Belanda, Norwegia, Rumania, Rusia, Swedia, Swiss, Belgia, Inggris, Kroasia, Denmark, Prancis, Jerman, Islandia, Portugal dan Spanyol. Sinovac, vaksin yang paling banyak digunakan secara global dengan 2,3 miliar dosis telah didistribusikan, mengatakan sedang saat ini sedang menguji kemampuan vaksinnya terhadap varian Omicron dalam sejumlah penelitian laboratorium.

Analisis Sentiment :positive

Dengan Sc


Summary dari Berita ID : 2451 (hasil teks yang diringkas):

Jakarta, CNBC Indonesia - Kementerian Kesehatan Singapura kembali menemukan dua kasus Covid-19 yang diduga Varian Omicron, satu diantaranya ditemukan pada pekerja di Bandara Changi.

Analisis Sentiment :positive

Dengan Score Sentiment :12
\Oringinal Article (Teks Asli):

Jakarta, CNBC Indonesia - Kementerian Kesehatan Singapura kembali menemukan dua kasus Covid-19 yang diduga Varian Omicron, satu diantaranya ditemukan pada pekerja di Bandara Changi.
Kementerian Kesehatan Singapura juga mengkonfirmasi dua warga yang positif tersebut sudah menerima dua dosis vaksin dan juga sudah menerima booster vaksin.
Simak informasinya dalam Program Profit, Jumat 10/12/2021 berikut ini

Summary dari Berita ID : 2452 (hasil teks yang diringkas):

Apalagi jika dibandingkan dengan gelombang Covid-19 sebelumnya yang menyebar di Afsel, di mana 2/3 pasien mengalami gejala parah. "Itu (vaksin) masih bisa menetralkan virus ini tetapi pada tingkat 


Summary dari Berita ID : 2455 (hasil teks yang diringkas):

Jakarta, CNBC Indonesia - Pemerintah mengatur sejumlah persyaratan bagi masyarakat yang ingin melakukan perjalanan jarak jauh ke luar daerah. Keputusan tersebut dituangkan dalam Instruksi Menteri Dalam Negeri (Inemdagri) 66/2021 tentang Pencegahan dan Penanggulangan Corona Virus Disease 2019 Pada Saat Natal Tahun 2021 dan Tahun Baru 2022.

Analisis Sentiment :positive

Dengan Score Sentiment :15
\Oringinal Article (Teks Asli):

Jakarta, CNBC Indonesia - Pemerintah mengatur sejumlah persyaratan bagi masyarakat yang ingin melakukan perjalanan jarak jauh ke luar daerah.
Keputusan tersebut dituangkan dalam Instruksi Menteri Dalam Negeri (Inemdagri) 66/2021 tentang Pencegahan dan Penanggulangan Corona Virus Disease 2019 Pada Saat Natal Tahun 2021 dan Tahun Baru 2022.
Bagi masyarakat yang ingin melakukan perjalanan jarak jauh yang menggunakan alat transportasi umum, ada beberapa persyaratan yang harus dipenuhi sebagai berikut: 1.
W

...


Summary dari Berita ID : 3058 (hasil teks yang diringkas):

Obat racikan Paxlovid terdiri dari dua tablet nirmatrelvir antivirus dan satu tablet ritonavir, obat yang telah lama digunakan sebagai zat penguat dalam rejimen HIV. Masalahnya, obat-obatan yang menimbulkan risiko interaksi banyak diresepkan untuk orang-orang yang paling berisiko terkena Covid-19 dengan kondisi kesehatan yang mendasarinya. Bukan hanya obat antivirus yang menjanjikan untuk pengobatan Covid-19. Namun, FDA mencabut izin Lilly untuk penggunaan monoterapi bamlanivimab—salah satu antibodi dalam koktail dengan etesevimab—karena tidak efektif terhadap varian SARS-CoV-2 yang muncul. Sedangkan Paxlovid, jenis antivirus berbeda yang dikenal sebagai protease inhibitor, tidak menunjukkan tanda-tanda "interaksi DNA mutagenik" seperti itu, kata Pfizer. Tetapi itu tidak berarti obat dari Pfizer tidak akan memiliki keunggulan di pasar. Atea sejak itu mengumumkan kemitraan strategis antara perusahaan akan dihentikan. Baca juga


Summary dari Berita ID : 3060 (hasil teks yang diringkas):

Baca juga:  Omicron Capai 46 Kasus di Indonesia, Pemerintah Perketat Pengawasan Pintu Masuk Selain itu, berdasarkan studi yang dilakukan di Inggris varian Omicron menyebabkan resiko perawatan di rumah sakit yang juga jauh lebih rendah dari varian Delta. "Saat ini jumlah kasus di Afrika Selatan juga sudah menunjukkan tren flattening," pungkasnya.

Analisis Sentiment :positive

Dengan Score Sentiment :5
\Oringinal Article (Teks Asli):

Laporan Wartawan Tribunnews Taufik Ismail  TRIBUNNEWS.
COM, JAKARTA - Menteri Koordinator Bidang Maritim dan Investasi Luhut Binsar Pandjaitan mengatakan bahwa secara global kasus Omicron terus meluas.
Varian Omicron yang awalnya ditemukan di Afrika Selatan tersebut kini sudah masuk ke 115 negara.
"Dengan total kasus mencapai lebih dari 184 ribu," kata Luhut dalam Konferensi pers virtual yang disiarkan Youtube Sekretariat Presiden, Senin, (27/12/2021).
Luhut mengatakan berdasarkan data di negara 


Summary dari Berita ID : 3064 (hasil teks yang diringkas):

Untuk diketahui, Yohanes Raga Imung adalah anggota DPRD Kabupaten Sikka dari Fraksi PKB. Setelah mendapatkan pukulan dari dua pemuda itu, Gaudensius bersama almarhum dan dua orang perempuan itu melanjutkan perjalanan. Namun dia tidak bisa dilantik karena mengalami gangguan kesehatan. Dia mengikuti kontestasi Pemilu 2019, dan bertarung pada Daerah Pemilihan Sikka 3 yang meliputi Kecamatan Talibura, Waigete, Bola, Waiblama, Doreng, dan Mapitara. Saat rekan-rekannya dilantik, Yohanes Raga Imung sedang terbaring di RSUD TC Hillers Maumere. Almarhum juga menjabat sebagai Sekretaris Fraksi PKB.

Analisis Sentiment :positive

Dengan Score Sentiment :4
\Oringinal Article (Teks Asli):

MAUMERE - Kabar duka menyelimuti Partai Kebangkitan Bangsa (PKB) Kabupaten Sikka.
Pasalnya, salah satu anggota Fraksi PKB DPRD Sikka, Yohanes Raga Imung meninggal dunia pada Selasa (28/12) malam sekitar pukul 21.00 WITA di Puskesmas Paga, Kecamatan Paga


Summary dari Berita ID : 3066 (hasil teks yang diringkas):

Di sisi lain, ia juga berharap agar pelaku usaha di Kota Solo dalam hal ini UMKM tidak hanya jadi reseller atau penjual, tetapi sebagai produsen. "Kemarin saya tegur itu Gojek, delivery fee-nya jangan kemahalan. Jadi saya enggak takut ditinggal mereka, tapi tidak menutup kemungkinan kolaborasi dengan mereka," ujarnya. Kita mau bekerja sama tapi jangan memberatkan kita," kata Gibran dalam Seminar Nasional bertajuk Tantangan Milenial Merebut Peluang Akses Pembiayaan Dalam Ekosistem UMKM dan Ekonomi Hijau yang disiarkan secara virtual, Selasa (28/12).

Analisis Sentiment :positive

Dengan Score Sentiment :1
\Oringinal Article (Teks Asli):

Wali Kota Surakarta Gibran Rakabuming Raka mengaku menegur Gojek baru-baru ini.
Ia meminta Gojek menurunkan ongkos kirim (ongkir) agar UMKM terbantu, terutama yang bergerak di bidang usaha makanan dan minuman.
"Kemarin saya tegur itu Gojek, delivery fee-nya jangan kemahalan.
Kalau kemahalan la


Summary dari Berita ID : 3069 (hasil teks yang diringkas):

Dengan tambahan itu, kini jumlah kasus positif COVID-19 di AS menjadi 54.090.539 orang. Meski demikian, sejauh ini varian Omicron menyebabkan gejala yang lebih ringan atau tanpa gejala pada mereka yang sudah menerima vaksinasi dosis penuh atau booster. Berdasarkan data worldometers, kematian harian akibat COVID-19 di AS juga kembali melesat. Antrean tampak mengular di banyak sentra pemeriksaan.

Analisis Sentiment :positive

Dengan Score Sentiment :6
\Oringinal Article (Teks Asli):

Penularan COVID-19 di Amerika Serikat semakin tidak karuan jelang berakhirnya tahun 2021.
Tercatat pada Selasa (28/12), kasus harian bertambah 267.199 orang.
Dengan tambahan itu, kini jumlah kasus positif COVID-19 di AS menjadi 54.090.539 orang.
Sedangkan kasus aktif jadi 11.976.405.
Berdasarkan data worldometers, kematian harian akibat COVID-19 di AS juga kembali melesat.
Dalam 24 jam, bertambah 1.515 pasien COVID-19 meninggal sehingga kini menja

In [ ]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="db-berita-mca"  
)
mycursor = mydb.cursor()
mycursor.execute("SELECT path FROM tb_source_sentiment WHERE lang = 'id' AND model= 'Lexicon' AND sentiment = 'negative'")
myresult = mycursor.fetchall()
print(myresult)

In [ ]:
|